In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### Whole - Daily

In [3]:
query = '''
    SELECT
        CusipId,
        TrdExctnDt,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDt,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM
            Trace_filteredWithRatings
        WHERE
            CntraMpId = 'C'
        GROUP BY
            CusipId,
            TrdExctnDt
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDt,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-daily.csv', index=False)

### Whole - Weekly

In [4]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtSOW,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtSOW,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                DATEADD(DAY, 1 - DATEPART(WEEKDAY, TrdExctnDt) + 1, TrdExctnDt) AS TrdExctnDtSOW
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtSOW
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtSOW,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-weekly.csv', index=False)

### Whole - Monthly

In [5]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtEOM,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtEOM,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                EOMONTH(TrdExctnDt) AS TrdExctnDtEOM
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtEOM
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtEOM,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-monthly.csv', index=False)

### Whole - Yearly

In [6]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtYr,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtYr,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                YEAR(TrdExctnDt) AS TrdExctnDtYr
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtYr
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtYr,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-yearly.csv', index=False)

### Institutional - Daily

In [7]:
query = '''
    SELECT
        CusipId,
        TrdExctnDt,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDt,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM
            Trace_filteredWithRatings
        WHERE
            CntraMpId = 'C'
            AND EntrdVolQt >= 500000
        GROUP BY
            CusipId,
            TrdExctnDt
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDt,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-daily-institutional.csv', index=False)

### Institutional - Weekly

In [8]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtSOW,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtSOW,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                DATEADD(DAY, 1 - DATEPART(WEEKDAY, TrdExctnDt) + 1, TrdExctnDt) AS TrdExctnDtSOW
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
                AND EntrdVolQt >= 500000
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtSOW
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtSOW,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-weekly-institutional.csv', index=False)

### Institutional - Monthly

In [9]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtEOM,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtEOM,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                EOMONTH(TrdExctnDt) AS TrdExctnDtEOM
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
                AND EntrdVolQt >= 500000
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtEOM
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtEOM,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-monthly-institutional.csv', index=False)

### Institutional - Yearly

In [10]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtYr,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtYr,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                YEAR(TrdExctnDt) AS TrdExctnDtYr
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
                AND EntrdVolQt >= 500000
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtYr
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtYr,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-yearly-institutional.csv', index=False)

### Retail - Daily

In [11]:
query = '''
    SELECT
        CusipId,
        TrdExctnDt,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDt,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM
            Trace_filteredWithRatings
        WHERE
            CntraMpId = 'C'
            AND EntrdVolQt < 250000
        GROUP BY
            CusipId,
            TrdExctnDt
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDt,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-daily-retail.csv', index=False)

### Retail - Weekly

In [12]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtSOW,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtSOW,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                DATEADD(DAY, 1 - DATEPART(WEEKDAY, TrdExctnDt) + 1, TrdExctnDt) AS TrdExctnDtSOW
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
                AND EntrdVolQt < 250000
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtSOW
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtSOW,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-weekly-retail.csv', index=False)

### Retail - Monthly

In [13]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtEOM,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtEOM,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                EOMONTH(TrdExctnDt) AS TrdExctnDtEOM
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
                AND EntrdVolQt < 250000
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtEOM
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtEOM,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-monthly-retail.csv', index=False)

### Retail - Yearly

In [14]:
query = '''
    SELECT
        CusipId,
        TrdExctnDtYr,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDtYr,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM (
            SELECT
                *,
                YEAR(TrdExctnDt) AS TrdExctnDtYr
            FROM
                Trace_filteredWithRatings
            WHERE
                CntraMpId = 'C'
                AND EntrdVolQt < 250000
        ) A
        GROUP BY
            CusipId,
            TrdExctnDtYr
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDtYr,
        CusipId
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/wermers-yearly-retail.csv', index=False)